In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

C:\Users\vijaya.sekhar\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


{'status': 'ok', 'restart': True}

**Colab Note:** On Colab, you need an [ngrok account](https://dashboard.ngrok.com/signup) to view some of the visualizations later. Please set up an account, then set your user token below:

In [2]:
NGROK_TOKEN = ""  # TODO: set your ngrok token if you are working on Colab

In [3]:
from zenml.environment import Environment

if Environment.in_google_colab():  # Colab only setup

    # install and authenticate ngrok
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

As an ML practitioner, you are probably familiar with building ML models using Scikit-learn, PyTorch, TensorFlow, or similar. An **[ML Pipeline](https://docs.zenml.io/starter-guide/pipelines)** is simply an extension, including other steps you would typically do before or after building a model, like data acquisition, preprocessing, model deployment, or monitoring. The ML pipeline essentially defines a step-by-step procedure of your work as an ML practitioner. Defining ML pipelines explicitly in code is great because:
- We can easily rerun all of our work, not just the model, eliminating bugs and making our models easier to reproduce.
- Data and models can be versioned and tracked, so we can see at a glance which dataset a model was trained on and how it compares to other models.
- If the entire pipeline is coded up, we can automate many operational tasks, like retraining and redeploying models when the underlying problem or data changes or rolling out new and improved models with CI/CD workflows.

Having a clearly defined ML pipeline is essential for ML teams that aim to serve models on a large scale.

## ZenML Setup
Throughout this series, we will define our ML pipelines using [ZenML](https://github.com/zenml-io/zenml/). ZenML is an excellent tool for this task, as it is straightforward and intuitive to use and has [integrations](https://docs.zenml.io/component-gallery/integrations) with most of the advanced MLOps tools we will want to use later. Make sure you have ZenML installed (via `pip install zenml`). Next, let's run some commands to make sure you start with a fresh ML stack.

In [ ]:
!rm -rf .zen
!zenml init

In [ ]:
import os
print(os.getcwd())

In [12]:
import numpy as np
import pandas as pd
from sklearn.base import ClassifierMixin
from sklearn.base import RegressorMixin
from sklearn.preprocessing import LabelEncoder
import json
from pandas.io.json import json_normalize
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import train_test_split


def train_test() -> None:
    """Train and test a Scikit-learn XGB Regressor"""
#     digits = load_digits()
#     data = digits.images.reshape((len(digits.images), -1))
    with open('C:/Users/vijaya.sekhar/Downloads/download/download/EVRI/EVRI/data/evri-nonprod-dgw-firehose-tracking-5-2022-05-05-06 (early morning sample).json') as f:
        ps_data = json.load(f)
    ps_df = pd.json_normalize(ps_data)
    df = ps_df[['originalParcelCreationTime',
   'trackingEvent.eventCategory',
  'trackingEvent.client.clientId',
   'trackingEvent.preadviceDetail.deliveryDetails.parcelId',
   'trackingEvent.preadviceDetail.parcelDetails.numberOfItems',
   'trackingEvent.preadviceDetail.parcelType.parcelTypeId']].copy()
        
    df["originalParcelCreationTime"]= pd.to_datetime(df["originalParcelCreationTime"])
    df['hour'] = df["originalParcelCreationTime"].dt.hour
    df['weekdayind'] = np.where(df['originalParcelCreationTime'].dt.dayofweek.isin([5,6]), 1, 0)
    b = [0,4,8,12,16,20,24]
    l = ['Late_Night', 'Early_Morning','Morning','Noon','Eve','Night']
    df['sessionofday'] = pd.cut(df['hour'], bins=b, labels=l, include_lowest=True)
    
    
    le = LabelEncoder()
    df['trackingEvent.eventCategory'] = le.fit_transform(df['trackingEvent.eventCategory'])
    df['sessionofday'] = le.fit_transform(df['sessionofday'])
    
    features = df[['trackingEvent.eventCategory', 'trackingEvent.client.clientId',
       'trackingEvent.preadviceDetail.deliveryDetails.parcelId','trackingEvent.preadviceDetail.parcelType.parcelTypeId','hour','sessionofday','weekdayind']]
    target = df[['trackingEvent.preadviceDetail.parcelDetails.numberOfItems']]
    train_size = int(len(df)*0.8)
    X_train,y_train = features[:train_size],target[:train_size]
    X_test,y_test = features[train_size:],target[train_size:]
    params = { 
     'objective': 'reg:squarederror',
     'max_depth': 3,
    'learning_rate': 0.1,
     'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_estimators': 100
    }
#     test_df = df[df['week'] >= '2019-06-01']
#     train_df = df[df['week'] < '2019-06-01']
#     X_train, y_train = train_df[features], train_df[label]
#     X_test, y_test = test_df[features], test_df[label]
    model = xgb.XGBRegressor(**params)
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test,pred)
    print(mse)
    df['pred'] = pd.DataFrame(pred)
#     test_acc = model.score(test_features,test_target)
#     print(f"Test accuracy: {test_acc}")


train_test()

2.6284396881237626e-10


## Turning experiments into ML pipelines with ZenML

In [16]:
from zenml.steps import step, Output


@step
def importer() -> Output(
#     X_train=np.ndarray,
#     X_test=np.ndarray,
#     y_train=np.ndarray,
#     y_test=np.ndarray,
    X_train=pd.DataFrame,
    X_test=pd.DataFrame,
    y_train=pd.DataFrame,
    y_test=pd.DataFrame,
):
    """Load the digits dataset as numpy arrays."""
#     digits = load_digits()
#     data = digits.images.reshape((len(digits.images), -1))
#     X_train, X_test, y_train, y_test = train_test_split(
#         data, digits.target, test_size=0.2, shuffle=False

    with open('C:/Users/vijaya.sekhar/Downloads/download/download/EVRI/EVRI/data/evri-nonprod-dgw-firehose-tracking-5-2022-05-05-06 (early morning sample).json') as f:
        ps_data = json.load(f)
    ps_df = pd.json_normalize(ps_data)
#     df = ps_df[['originalParcelCreationTime','trackingEvent.eventCategory','trackingEvent.client.clientId','trackingEvent.preadviceDetail.deliveryDetails.parcelId','trackingEvent.preadviceDetail.parcelDetails.numberOfItems','trackingEvent.preadviceDetail.parcelType.parcelTypeId']].copy()
    
    df = ps_df.filter(['originalParcelCreationTime','trackingEvent.eventCategory','trackingEvent.client.clientId','trackingEvent.preadviceDetail.deliveryDetails.parcelId','trackingEvent.preadviceDetail.parcelDetails.numberOfItems','trackingEvent.preadviceDetail.parcelType.parcelTypeId'], axis =1)
    
    df["originalParcelCreationTime"]= pd.to_datetime(df["originalParcelCreationTime"])
    df['hour'] = df["originalParcelCreationTime"].dt.hour
    df['weekdayind'] = np.where(df['originalParcelCreationTime'].dt.dayofweek.isin([5,6]), 1, 0)
    b = [0,4,8,12,16,20,24]
    l = ['Late_Night', 'Early_Morning','Morning','Noon','Eve','Night']
    df['sessionofday'] = pd.cut(df['hour'], bins=b, labels=l, include_lowest=True)
    
    
    le = LabelEncoder()
    df['trackingEvent.eventCategory'] = le.fit_transform(df['trackingEvent.eventCategory'])
    df['sessionofday'] = le.fit_transform(df['sessionofday'])
    
    features = df[['trackingEvent.eventCategory', 'trackingEvent.client.clientId',
       'trackingEvent.preadviceDetail.deliveryDetails.parcelId','trackingEvent.preadviceDetail.parcelType.parcelTypeId','hour','sessionofday','weekdayind']]
    target = df[['trackingEvent.preadviceDetail.parcelDetails.numberOfItems']]
    train_size = int(len(df)*0.8)
    X_train,y_train = features[:train_size],target[:train_size]
    X_test,y_test = features[train_size:],target[train_size:]
#     X_test.to_numpy()
#     y_test.to_numpy()
#     X_train.to_numpy()
#     y_train.to_numpy()
    return X_train, X_test, y_train, y_test


@step
def xg_trainer(
#     X_train: np.ndarray,
#     y_train: np.ndarray,
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
) -> RegressorMixin:
    """Train an sklearn XGB Regressor."""
#     model = SVC(gamma=0.001)
#     model.fit(X_train, y_train)
    params = { 
     'objective': 'reg:squarederror',
     'max_depth': 3,
    'learning_rate': 0.1,
     'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_estimators': 100
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train,y_train)

    return model


@step
def evaluator(
#     X_test: np.ndarray,
#     y_test: np.ndarray,
    X_test: pd.DataFrame,
    y_test: pd.DataFrame,
    
    model: RegressorMixin,
) -> float:
#     """Calculate the test set accuracy of an sklearn model."""
#     test_acc = model.score(X_test, y_test)
#     print(f"Test accuracy: {test_acc}")
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test,pred)
    print(mse)
    #df['pred'] = pd.DataFrame(pred)
    return mse

Similarly, we can use ZenML's `@pipeline` decorator to connect all of our steps into an ML pipeline.

Note that the pipeline definition does not depend on the concrete step functions we defined above; it merely establishes a recipe for how data moves through the steps. This means we can replace steps as we wish, e.g., to run the same pipeline with different models to compare their performances.

In [17]:
from zenml.pipelines import pipeline


@pipeline
def xg_pipeline(importer, trainer, evaluator):
    """Links all the steps together in a pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

## Running ZenML Pipelines
Finally, we initialize our pipeline with concrete step functions and call the `run()` method to run it.

In [18]:
xg_pipeline = xg_pipeline(
    importer=importer(), trainer=xg_trainer(), evaluator=evaluator(),
#     model_register=mlflow_register_model_step(
#             params=MLFlowRegistryParameters(
#             name="xgregression-model",
#             description="A xg model trained with ZenML",
#             tags={"framework": "xgboost", "dataset": "data"},
#             description=f"The 1st run of the mlflow_training_pipeline")
#     )
)
xg_pipeline.run()

Registered pipeline xg_pipeline (version 3).
Running pipeline xg_pipeline on stack default (caching enabled)
Step importer has started.
Using cached version of importer.
Step trainer has started.
Using cached version of trainer.
Step evaluator has started.
2.6284396881237626e-10
Step evaluator has finished in 0.174s.
Pipeline run xg_pipeline-2023_03_29-08_50_28_101584 has finished in 2.407s.
Pipeline visualization can be seen in the ZenML Dashboard. Run zenml up to see your pipeline!


In [ ]:
!zenml up

And that's it, we just built and ran our first ML pipeline! Great job!

You can now visualize the pipeline run in ZenML's dashboard. To do so, run 
`zenml up` to spin up a ZenML dashboard locally, log in with username `default`
and empty password, and navigate to the "Runs" tab in the "Pipelines" section.

In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")
        !zenml up --blocking --port {port}

    else:
        !zenml up --blocking --port {port}

start_zenml_dashboard()

In [ ]:
Environment.

In [ ]:
# import mlflow
# # zenml integration install mlflow -y
# # Register the MLflow experiment tracker
# # zenml experiment-tracker register mlflow_experiment_tracker --flavor=mlflow

# # Register and set a stack with the new experiment tracker
# # zenml stack register custom_stack -e mlflow_experiment_tracker ... --set


# @step(experiment_tracker="<MLFLOW_TRACKER_STACK_COMPONENT_NAME>")
# def tf_trainer(
#     x_train: pd.DataFrame,
#     y_train: pd.DataFrame,
# ) -> tf.keras.Model:
#     """Train a neural net from scratch to recognize MNIST digits return our
#     model or the learner"""
    
#     # compile model

#     mlflow.xgboost.autolog()
    
#     # train model

#     # log additional information to MLflow explicitly if needed

#     mlflow.log_param(...)
#     mlflow.log_metric(...)
#     mlflow.log_artifact(...)

#     return model

In [ ]:
# from zenml.integrations.mlflow.steps.mlflow_deployer import (
#     MLFlowDeployerParameters,
#     mlflow_model_registry_deployer_step,
# )

# mlflow_training_pipeline(
#     importer=importer(),
#     trainer=xg_trainer(params=params),
#     evaluator=xg_evaluator(),
#     model_register=mlflow_register_model_step(
#         params=MLFlowRegistryParameters(
#             name="xgregression-model",
#             description="A xg model trained with ZenML",
#             tags={"framework": "xgboost", "dataset": "data"},
#             description=f"The 1st run of the mlflow_training_pipeline.",
#         )
#     ),
# ).run()

# # The list of parameters that can be passed to the mlflow_model_register_step:
# """
#     name: Name of the registered model.
#     experiment_name: Name of the MLflow experiment to be used for the run.
#     run_name: Name of the MLflow run to be created.
#     run_id: ID of the MLflow run to be used.
#     model_source_uri: URI of the model source. If not provided, the model
#         will be fetched from the MLflow tracking server.
#     description: Description of the model.
#     metadata: Tags to be added to the model.
# """